In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import lxml
import pandas as pd

In [214]:
data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller20081_202312_1000.csv")
urls = data2['detail_link']
urls = list(set(urls))
len(urls)

C:\Users\hi\AppData\Local\Temp\ipykernel_5668\3649023135.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller20081_202312_1000.csv")


42780

In [225]:
book_id = urls[0].split("/")[-1]

88414739


In [143]:
# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

In [227]:
# 리뷰 1페이지 내용 수집 함수
def review_unpack(book_id, html_source):
    soup = bs(html_source, "lxml")
    reviews = soup.select("div.reviewInfoGrp.lnkExtend")
    for review in reviews:
        all_reviews.append([book_id, review.select_one("div.reviewInfoBot.origin > div.review_cont").text])               

In [234]:
range(1)

range(0, 1)

In [ ]:
for d_url in urls:
    # driver.get("https://www.yes24.com/Product/Goods/122120495")  # 크롤링 할 웹사이트 주소
    driver.get(d_url)
    book_id = d_url.split("/")[-1] # 책 id
    try:
        driver.find_element(By.CSS_SELECTOR, "li#total > a").click()
        time.sleep(3)
        driver.find_element(By.CSS_SELECTOR, "li#total > a").click()
    except Exception as e:
        print(e)
        continue
    # 전체 리뷰 갯수
    n_reviews = driver.find_element(By.CSS_SELECTOR, "li#total[data-review-count]").text.split()[2].strip("()")  
    n_review_pages = int(n_reviews) // 5 + 1 # 전체 리뷰 페이지 수
    n_next_page = n_review_pages // 10  # 전체 클릭할 넥스트 버튼 수
    print("전체 리뷰 갯수: ", n_reviews, ", 전체 리뷰 페이지 수: ", n_review_pages, ", 전체 클릭할 넥스트 버튼 수: ", n_next_page, end="\r")
    
    # 전체 리뷰 수집 코드
    all_reviews = [] # 책 1권의 리뷰 모두 저장
    if n_review_pages <= 10:  # 리뷰 10개 이하
        # 전체리뷰 처음 클릭시 나오는 리뷰 수집
        review_unpack(book_id, driver.page_source)
        # 2-10페이지까지 수집
        for page in range(4, 14):
            try:
                driver.find_element(By.CSS_SELECTOR, f"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child({page})").click()
                time.sleep(6)
                review_unpack(book_id, driver.page_source)
                print(f"현재 진행 페이지:{book_id} {page}/{n_review_pages}", end="\r")
            except Exception as e:
                print(e)
                break
    else:
        for next in range(n_next_page+1):
    #         if next == 1:
            # 전체리뷰 처음 클릭시 나오는 리뷰 수집
            review_unpack(book_id, driver.page_source)
            # 2-10페이지까지 수집
            for page in range(4, 14):
                try:
                    driver.find_element(By.CSS_SELECTOR, f"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child({page})").click()
                    time.sleep(6)
                    review_unpack(book_id, driver.page_source)
                    print(f"현재 진행 페이지:{book_id} {page}/{next}/{n_next_page}", end="\r")
                except Exception as e:
                    print(e, end="\r")
                    break
            try:
                driver.find_element(By.CSS_SELECTOR, "a.bgYUI.next").click() # next 버튼 클릭
                time.sleep(5)
            except Exception as e:
                print(e)
    book_review = pd.DataFrame(all_reviews, columns=['book_id', 'review'])
    book_review.to_csv(f"./data/yes24/{book_id}_reviews.csv")

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(7)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(12)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(6)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(6)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: element click intercepted: Element <a href="javascript:void(0);" class="bgYUI next dim">...</a> is not clickable at point (461, 13). Other element would receive the click: <h2 class="gd_name">...</h2>
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008C07AB]
	(No symbol) [0x008BF2AF]
	(No symbol) [0x008BD89B]
	(No symbol) [0x008BCBA1]
	(No symbol) [0x008B4FC0]
	(No symbol) [0x008D7FDC]
	(No symbol) [0x008B4A4E]
	(No symbol) [0x008D8254]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [

...

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(7)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(8)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(8)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(6)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008BB940]
	(No symbol) [0x008BBE0B]
	(No symbol) [0x008ED1F2]
	(No symbol) [0x008D8024]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B67C6

Message: element click intercepted: Element <a href="javascript:void(0);" class="bgYUI next dim">...</a> is not clickable at point (461, 13). Other element would receive the click: <h2 class="gd_name">...</h2>
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00A072A3+45731]
	(No symbol) [0x00992D51]
	(No symbol) [0x0088880D]
	(No symbol) [0x008C07AB]
	(No symbol) [0x008BF2AF]
	(No symbol) [0x008BD89B]
	(No symbol) [0x008BCBA1]
	(No symbol) [0x008B4FC0]
	(No symbol) [0x008D7FDC]
	(No symbol) [0x008B4A4E]
	(No symbol) [0x008D8254]
	(No symbol) [0x008EB7A2]
	(No symbol) [0x008D7DD6]
	(No symbol) [0x008B31F6]
	(No symbol) [0x008B439D]
	GetHandleVerifier [0x00D10716+3229462]
	GetHandleVerifier [0x00D584C8+3523784]
	GetHandleVerifier [0x00D5214C+3498316]
	GetHandleVerifier [0x00A91680+611968]
	(No symbol) [0x0099CCCC]
	(No symbol) [0x00998DF8]
	(No symbol) [0x00998F1D]
	(No symbol) [0x0098B2C7]
	BaseThreadInitThunk [0x767AFCC9+25]
	RtlGetAppContainerNamedObjectPath [

C:\Users\hi\AppData\Local\Temp\ipykernel_5668\3369858076.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller20081_202312_1000.csv")


In [207]:
len(urls)

187964

In [209]:
len(list(set(urls)))

42780